In [1]:
import pandas as pd
import numpy as np

# Notes:

You are banned from using loops (`for` or `while` or any other) for this entire workshop!

You shouldn't be using loops almost ever with pandas in any case, so break out of the habit now.

## 1. DataFrame basics


Consider the following Python dictionary `data` and Python list `labels`:

``` python
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
```
(This is just some meaningless data I made up with the theme of animals and trips to a vet.)

**1.** Create a DataFrame `df` from this dictionary `data` which has the index `labels`.

**2.** Select only the rows where visits are 3 or more. Which types of animals are these?

**3.** Select the rows where visists are 3 and the animal is a cat

**4.** Calculate the sum of all visits in `df` (i.e. the total number of visits).

**5.** Calculate the mean age for each different animal in `df`.

**6.** Append a new row 'k' to `df` with your choice of values for each column. Then delete that row to return the original DataFrame.



In [2]:
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

#1
df = pd.DataFrame(data, index = labels)

#2
df.loc[df.visits >= 3,:]

#3
df.loc[(df.visits >= 3) & (df.animal == 'cat'),:]

#4
df['visits'].sum()

#5
df['age'].mean()

#6
s = pd.Series(['owl', 10, 5, 'yes'], index = ['animal', 'age', 'visits', 'priority'], name = 'k')
df = df.append(s)
df.drop('k', axis = 0)

,animal,age,visits,priority
a,cat,2.5,1,yes
b,cat,3.0,3,yes
c,snake,0.5,2,no
d,dog,NaN,3,yes
e,dog,5.0,2,no
f,cat,2.0,3,no
g,snake,4.5,1,no
h,cat,NaN,1,yes
i,dog,7.0,2,no
j,dog,3.0,1,no


# 2.1 Shifty problem

You have a DataFrame `df` with a column 'A' of integers. For example:
```python
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
```

How do you filter out rows which contain the same integer as the row immediately above?

You should be left with a column containing the following values:

```python
1, 2, 3, 4, 5, 6, 7
```

### Hint: use the `shift()` method

In [3]:
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
df.loc[
    df.A.shift(periods = -1) != df.A
]

,A
0,1
2,2
3,3
4,4
7,5
8,6
10,7


# 2.2 columns sum min

Suppose you have DataFrame with 10 columns of real numbers, for example:

```python
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))
```
Which column of numbers has the smallest sum? Return that column's label.

In [4]:
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))
df.sum().idxmin()

'b'

# 2.3 Duplicates

How do you count how many unique rows a DataFrame has (i.e. ignore all rows that are duplicates)?

**hint:** There's a method for to find duplicate rows for you

In [5]:
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [3, 3, 0.5, 5, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 1, 2, 2, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no']}

"""
Using same data from question 1 but changed some of points to have duplicates 
because I was too lazy to come up with a new dataset
"""

df = pd.DataFrame(data)
df = df.drop_duplicates()
df

,animal,age,visits,priority
0,cat,3.0,1,yes
2,snake,0.5,2,no
3,dog,5.0,2,yes
5,cat,2.0,3,no
6,snake,4.5,1,no
7,cat,NaN,1,yes
8,dog,7.0,2,no
9,dog,3.0,1,no


# 2.4 Group Values

A DataFrame has a column of groups 'grps' and and column of integer values 'vals': 

```python
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
```
For each *group*, find the sum of the three greatest values.  You should end up with the answer as follows:
```
grps
a    409
b    156
c    345
```

In [6]:
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
dfa = df.loc[df.grps == 'a'].nlargest(3,'vals')
dfb = df.loc[df.grps == 'b'].nlargest(3,'vals')
dfc = df.loc[df.grps == 'c'].nlargest(3,'vals')

df = pd.DataFrame({'grps': list('abc'),
                   'vals': [dfa['vals'].sum(),dfb['vals'].sum(),dfc['vals'].sum()]})
df

,grps,vals
0,a,409
1,b,156
2,c,345


# 3. Cleaning Data

### Making a DataFrame easier to work with

It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

```python
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})
```

Formatted, it looks like this:

```
            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis           NaN          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"
```

**1.** Some values in the the **FlightNumber** column are missing (they are `NaN`). These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Modify `df` to fill in these missing numbers and make the column an integer column (instead of a float column).

In [7]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})
df = df.interpolate(method = 'linear', limit_direction = 'forward')
df.FlightNumber = df.FlightNumber.astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


# 3.2 column splitting

The **From\_To** column would be better as two separate columns! Split each string on the underscore delimiter `_` to make two new columns `From` and `To` to your dataframe.

In [8]:
df[['From','To']] = df.From_To.str.split("_", expand = True)
df = df.drop(['From_To'], axis = 1)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),LoNDon,paris
1,10055,[],<Air France> (12),MAdrid,miLAN
2,10065,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,10075,[13],12. Air France,Budapest,PaRis
4,10085,"[67, 32]","""Swiss Air""",Brussels,londOn


# 3.3 Clean Text

Make the text in your dataframe:

- From and To columns should be lowercase with only first letter capitalized

- In the **Airline** column, you can see some extra puctuation and symbols have appeared around the airline names. Pull out just the airline name. E.g. `'(British Airways. )'` should become `'British Airways'`.

In [9]:
df['From'] = df['From'].str.capitalize()
df['To'] = df['To'].str.capitalize()
df['Airline'] = df['Airline'].str.replace('[^\w\s]','')
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM,London,Paris
1,10055,[],Air France 12,Madrid,Milan
2,10065,"[24, 43, 87]",British Airways,London,Stockholm
3,10075,[13],12 Air France,Budapest,Paris
4,10085,"[67, 32]",Swiss Air,Brussels,London


# Exercise 4.1: Column Splitting

Given the unemployment data in `data/country_total.csv`, split the `month` column into two new columns: a `year` column and a `month` column, both integers

In [10]:
df = pd.read_csv('data/country_total.csv')
df[['year','month']] = df.month.astype(str).str.split(".", expand = True)
df.drop(['month'], axis = 1)
df.month.astype(int)
df.year.astype(int)
df

,country,seasonality,month,unemployment,unemployment_rate,year
0,at,nsa,01,171000,4.5,1993
1,at,nsa,02,175000,4.6,1993
2,at,nsa,03,166000,4.4,1993
3,at,nsa,04,157000,4.1,1993
4,at,nsa,05,147000,3.9,1993
...,...,...,...,...,...,...
20791,uk,trend,06,2429000,7.7,2010
20792,uk,trend,07,2422000,7.7,2010
20793,uk,trend,08,2429000,7.7,2010
20794,uk,trend,09,2447000,7.8,2010


# 4.2 Group Statistics

Given the unemployment data in `data/country_sex_age.csv`, give the average unemployment rate for:

- Each gender
- Each Age Group
- Both Together

**HINT:** The `seasonality` column makes it such that the data is repeated for each method of calculating unemployment (`nsa`, `trend`, etc.). Can you ignore this and group over it? Or should you take the average for each?

In [11]:
df = pd.read_csv('data/country_sex_age.csv')
f = df.loc[df.sex == 'f', 'unemployment_rate'].mean()
m = df.loc[df.sex == 'm', 'unemployment_rate'].mean()
y1 = df.loc[df.age_group == 'y25-74', 'unemployment_rate'].mean()
y2 = df.loc[df.age_group == 'y_lt25', 'unemployment_rate'].mean()
fy1 = df.loc[(df.sex == 'f') & (df.age_group == 'y25-74'), 'unemployment_rate'].mean()
fy2 = df.loc[(df.sex == 'f') & (df.age_group == 'y_lt25'), 'unemployment_rate'].mean()
my1 = df.loc[(df.sex == 'm') & (df.age_group == 'y25-74'), 'unemployment_rate'].mean()
my2 = df.loc[(df.sex == 'm') & (df.age_group == 'y_lt25'), 'unemployment_rate'].mean()

print(f, m, y1, y2, fy1, fy2, my1, my2)

12.982629178569676 11.671025871093027 6.905393549673485 17.774056532852356 7.566771347390394 18.457435122196987 6.244015751956556 17.098035990229665


# 4.3 Estimating group size

Given that we have the unemployment **rate** as a % of total population, and the number of total unemployed, we can estimate the total population.

Give an estimate of the total population for men and women in each age group.

Does this change depending on the unemployment seasonality calculation method?

In [12]:
df['population'] = df.apply(lambda row: row.unemployment / (row.unemployment_rate/100), axis = 1)
pfy1 = df.loc[(df.sex == 'f') & (df.age_group == 'y25-74'), 'population'].mean()
pfy2 = df.loc[(df.sex == 'f') & (df.age_group == 'y_lt25'), 'population'].mean()
pmy1 = df.loc[(df.sex == 'm') & (df.age_group == 'y25-74'), 'population'].mean()
pmy2 = df.loc[(df.sex == 'm') & (df.age_group == 'y_lt25'), 'population'].mean()
nsa = df.loc[(df.seasonality == 'nsa'), 'population'].mean()
sa = df.loc[(df.seasonality == 'sa'), 'population'].mean()
trend = df.loc[(df.seasonality == 'trend'), 'population'].mean()

print(pfy1, pfy2, pmy1, pmy2)
print("nsa=",nsa, "sa=",sa, "trend=",trend)  ##the population changes

3220601.772863034 566582.5758164536 4357668.061171533 667982.6295767102
nsa= 2069864.667892437 sa= 2298883.8571608183 trend= 2251532.9234996643


# 5.1 Tennis

In `data/tennis.csv` you have games that Roger Federer played against various opponents. Questions:

1. How many games did Federer win?

2. What is Federer's win/loss ratio?

3. Who were Federer's top 5 opponents?

In [13]:
df = pd.read_csv('data/tennis.csv')
df


## 1
win = len(df[df['winner'] == 'Roger Federer'])
print(win, "games won")

## 2
ratio = len(df[df['winner'] == 'Roger Federer']) / len(df[df['winner'] != 'Roger Federer'])
print(ratio, "win/loss ratio")

## 3
top = df['opponent'].value_counts().head(6)
print(top[1:6]) ### 0th index prints out "Bye" which I assume are unplayed games?

972 games won
4.695652173913044 win/loss ratio
Novak Djokovic (SRB)       29
Rafael Nadal (ESP)         28
Lleyton Hewitt (AUS)       26
Andy Roddick (USA)         24
Nikolay Davydenko (RUS)    19
Name: opponent, dtype: int64


# 5.2 Over time

1. What was Federer's best year? In terms of money, and then in terms of number of wins

2. Did Federer get better or worse over time?

In [14]:
## 1
print("1.")
ymoney = df.loc[(df.winner == 'Roger Federer') & (df['tournament round'] == 'F'), ['year', 'tournament prize money']]
ymoney['tournament prize money'] = ymoney['tournament prize money'].str.replace('[^\w\s]','')
ymoney['tournament prize money'] = ymoney['tournament prize money'].str.replace('A','')
ymoney['tournament prize money'] = ymoney['tournament prize money'].astype(float)
print((ymoney.groupby('year').sum())['tournament prize money'].idxmax())

ywin = df.loc[(df.winner == 'Roger Federer'), 'year']
print(ywin.value_counts().idxmax())

## 2
print("2. Seems Federer got better over time but peaked in the mid 2000s")

1.
2007
2006
2. Seems Federer got better over time but peaked in the mid 2000s


# 5.3 Total money won

In the data, you'll find the `tournament round`, one value of which, `F` indicates the final.

Assuming Federer wins the money in the `tournament prize money` if he wins a final in a tournament, how much money has Federer made in tournaments in this dataset?

In [15]:
t_win = df.loc[(df.winner == 'Roger Federer') & (df['tournament round'] == 'F'), ['tournament prize money']]
t_win['tournament prize money'] = t_win['tournament prize money'].str.replace('[^\w\s]','')
t_win['tournament prize money'] = t_win['tournament prize money'].str.replace('A','')
t_win['tournament prize money'] = t_win['tournament prize money'].astype(float)
print("Roger's total winnings should be ${:,}".format(t_win['tournament prize money'].sum()))

Roger's total winnings should be $44,934,964.0
